IDS 472 Homework 5 04/13/25 

Completed by: Krima Mehta, Ihor Stepura, Artem Larin, and Arturo Perez

Q1a: CART & AdaBoost 5-fold CV

Q1a.i: Load and preprocess

In [42]:
from sklearn.datasets import fetch_california_housing

data = fetch_california_housing()

X, y = data.data, data.target

Q1a.ii: Train & evaluate 5-fold CV CART 

In [43]:
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import KFold, cross_val_score

from sklearn.metrics import make_scorer, mean_absolute_error, r2_score

import numpy as np

kf = KFold(n_splits=5, shuffle=True, random_state=42)

mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

r2_scorer = make_scorer(r2_score)

cart = DecisionTreeRegressor(min_samples_leaf=10, random_state=42)

mae_cart = -cross_val_score(cart, X, y, cv=kf, scoring=mae_scorer)

r2_cart = cross_val_score(cart, X, y, cv=kf, scoring=r2_scorer)

print("CART Mean MAE:", round(np.mean(mae_cart), 3))

print("CART Mean R^2:", round(np.mean(r2_cart), 3))


CART Mean MAE: 0.405
CART Mean R^2: 0.721


Q1a.iii: Train and evaluate 5-fold CV AdaBoost 

In [44]:
from sklearn.ensemble import AdaBoostRegressor

ada = AdaBoostRegressor(

    base_estimator=DecisionTreeRegressor(min_samples_leaf=10, random_state=42),

    n_estimators=50,

    random_state=42

)

mae_ada = -cross_val_score(ada, X, y, cv=kf, scoring=mae_scorer)

r2_ada = cross_val_score(ada, X, y, cv=kf, scoring=r2_scorer)

print("AdaBoost Mean MAE:", round(np.mean(mae_ada), 3))

print("AdaBoost Mean R^2:", round(np.mean(r2_ada), 3))


/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/opt/anaconda3/lib/python3.11/

AdaBoost Mean MAE: 0.319
AdaBoost Mean R^2: 0.828


Q1a.iv: Best model based on R^2

In [45]:
mean_r2_cart = np.mean(r2_cart)

mean_r2_ada = np.mean(r2_ada)

best_model = "AdaBoost" if mean_r2_ada > mean_r2_cart else "CART"

print("Best Model (Based on R^2):", best_model)


Best Model (Based on R^2): AdaBoost


Q1b: RF Grid Search & Evaluation

Q1b.i: Train-test split (80/20)

In [46]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(

    X, y, test_size=0.2, random_state=42
    
)


Q1b.ii: Grid search 10-Fold CV RF

In [47]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import GridSearchCV

param_grid = {

    'max_features': [X.shape[1] // 3],

    'min_samples_leaf': [2, 10],

    'n_estimators': [10, 50, 100]

}

rf = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(

    rf,

    param_grid,

    cv=10,

    scoring='r2',

    n_jobs=-1

)

grid_search.fit(X_train, y_train)

best_rf = grid_search.best_estimator_

print("Best Parameters:", grid_search.best_params_)

print("Best CV R^2:", round(grid_search.best_score_, 3))


Best Parameters: {'max_features': 2, 'min_samples_leaf': 2, 'n_estimators': 100}
Best CV R^2: 0.813


Q1b.iii: Best model on test set

In [48]:
y_pred = best_rf.predict(X_test)

test_r2 = r2_score(y_test, y_pred)

test_mae = mean_absolute_error(y_test, y_pred)

print("Test R^2:", round(test_r2, 3))

print("Test MAE:", round(test_mae, 3))


Test R^2: 0.807
Test MAE: 0.334
